### Imports

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import librosa
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, confusion_matrix
import tensorflow as tf
import os
import optuna
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Import dos dados

In [ ]:
!wget -O /content/drive/MyDrive/Trabalho_AC2/dataset.gz "https://goo.gl/8hY5ER"

--2024-11-21 10:09:31--  https://goo.gl/8hY5ER
Resolving goo.gl (goo.gl)... 172.217.7.46, 2607:f8b0:4025:811::200e
Connecting to goo.gl (goo.gl)|172.217.7.46|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz [following]
--2024-11-21 10:09:31--  https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.98.238, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1203745/files/UrbanSound8K.tar.gz [following]
--2024-11-21 10:09:32--  https://zenodo.org/records/1203745/files/UrbanSound8K.tar.gz
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 6023741708 (5.6G) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/Trabalho_AC2/dataset.gz’

/content/drive/MyDr 100

In [ ]:
!gunzip /content/drive/MyDrive/Trabalho_AC2/dataset.gz

gzip: /content/drive/MyDrive/Trabalho_AC2/dataset already exists; do you wish to overwrite (y or n)? y


In [ ]:
!tar -xvf /content/drive/MyDrive/Trabalho_AC2/dataset -C /content/drive/MyDrive/Trabalho_AC2/

A saída de streaming foi truncada nas últimas 5000 linhas.
UrbanSound8K/audio/fold4/17480-2-0-6.wav
UrbanSound8K/audio/fold4/17480-2-0-9.wav
UrbanSound8K/audio/fold4/175904-2-0-11.wav
UrbanSound8K/audio/fold4/175904-2-0-24.wav
UrbanSound8K/audio/fold4/176003-1-0-0.wav
UrbanSound8K/audio/fold4/176638-5-0-0.wav
UrbanSound8K/audio/fold4/177756-2-0-10.wav
UrbanSound8K/audio/fold4/177756-2-0-4.wav
UrbanSound8K/audio/fold4/177756-2-0-5.wav
UrbanSound8K/audio/fold4/177756-2-0-7.wav
UrbanSound8K/audio/fold4/179862-1-0-0.wav
UrbanSound8K/audio/fold4/180977-3-1-1.wav
UrbanSound8K/audio/fold4/180977-3-1-5.wav
UrbanSound8K/audio/fold4/183989-3-1-21.wav
UrbanSound8K/audio/fold4/183989-3-1-23.wav
UrbanSound8K/audio/fold4/185709-0-0-0.wav
UrbanSound8K/audio/fold4/185709-0-0-1.wav
UrbanSound8K/audio/fold4/185709-0-0-6.wav
UrbanSound8K/audio/fold4/185709-0-0-7.wav
UrbanSound8K/audio/fold4/185909-2-0-102.wav
UrbanSound8K/audio/fold4/185909-2-0-13.wav
UrbanSound8K/audio/fold4/185909-2-0-17.wav
UrbanSound

In [ ]:
metadata_path = '/content/drive/MyDrive/Trabalho_AC2/UrbanSound8K/metadata/UrbanSound8K.csv'

metadata = pd.read_csv(metadata_path)

metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [ ]:
labels = metadata['class'].unique()
print(labels)

['dog_bark' 'children_playing' 'car_horn' 'air_conditioner' 'street_music'
 'gun_shot' 'siren' 'engine_idling' 'jackhammer' 'drilling']


###Aplicação do Espectograma de Mel aos dados em áudio

Por tentativa/erro percebemos que um número de mels adequados para o tamanho dos áudios seria 32. Mais do que isso geraria filtros vazios. Para além disso, definimos a frequência máxima como o mínimo entre metade da taxa de amostragem e 8kHz para obtermos valores razoáveis

In [ ]:
def melspectrogram(file_path, target_length=376):
    audio, sr = librosa.load(file_path, sr=None)
    if audio.size == 0:
        raise ValueError(f"Arquivo de áudio vazio: {file_path}")

    fmax = min(8000, sr / 2)
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=32, fmax=fmax)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    current_length = mel_spec_db.shape[-1]

    if current_length < target_length:
        mel_spec_db = np.pad(mel_spec_db, ((0, 0), (0, target_length - current_length)), mode='constant')
    elif current_length > target_length:
        mel_spec_db = mel_spec_db[:, :target_length]

    if np.all(mel_spec_db == 0):
        raise ValueError(f"Espectrograma vazio para o arquivo: {file_path}")

    mel_spec_db = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min())

    return mel_spec_db

In [ ]:
def save_spectrogram(input_audio_path, spectrogram, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    file_name = os.path.splitext(os.path.basename(input_audio_path))[0] + '.npy'
    output_path = os.path.join(output_folder, file_name)

    np.save(output_path, spectrogram)

In [ ]:
def process_fold(input_folder, output_folder):
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.wav'):
            audio_path = os.path.join(input_folder, file_name)
            spectrogram = melspectrogram(audio_path)

            save_spectrogram(audio_path, spectrogram, output_folder)


def process_all_folds(audio_root):
    processed_root = "processed_audios"

    os.makedirs(processed_root, exist_ok=True)

    for fold in os.listdir(audio_root):
        fold_path = os.path.join(audio_root, fold)
        if os.path.isdir(fold_path):
            processed_fold_path = os.path.join(processed_root, fold)

            os.makedirs(processed_fold_path, exist_ok=True)

            process_fold(fold_path, processed_fold_path)

###Definição de X e y

In [ ]:
base_path = 'processed_audios'
metadata['slice_file_name'] = metadata['slice_file_name'].str.replace('.wav', '.npy')
metadata['file_path'] = metadata.apply(lambda row: os.path.join(base_path, f'fold{row["fold"]}', row['slice_file_name']), axis=1)

X = metadata['file_path'].values
y = metadata['classID'].values

print(X[:5])

['processed_audios/fold5/100032-3-0-0.npy'
 'processed_audios/fold5/100263-2-0-117.npy'
 'processed_audios/fold5/100263-2-0-121.npy'
 'processed_audios/fold5/100263-2-0-126.npy'
 'processed_audios/fold5/100263-2-0-137.npy']


In [ ]:
class AudioDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None, target_length=376):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
        self.target_length = target_length

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, index):
        file_path = self.file_paths[index]
        label = self.labels[index]

        try:
            features = np.load(file_path)
        except FileNotFoundError as e:
            print(e)
            features = np.zeros((32, 376))

        features = torch.tensor(features, dtype=torch.float32).unsqueeze(0)

        if self.transform:
            features = self.transform(features)

        return features, label

###Definição dos folds de acordo com os folds dos dados originais

Como estamos a definir os dados para a otimização bayesiana não vamos aplicar 10-cross-validation, por isso atribuimos 1 fold específico para teste, 1 fold específico para desenvolvimento e os restantes para treino

In [ ]:
folds = {i: [] for i in range(1, 11)}

for i in range(len(metadata)):
    fold_number = metadata.iloc[i]['fold']
    folds[fold_number].append(i)

def get_train_val_test_split(folds, test_fold, dev_fold):
    train_indices = []
    test_indices = folds[test_fold]
    dev_indices = folds[dev_fold]

    for fold in folds:
        if fold != test_fold and fold != dev_fold:
            train_indices.extend(folds[fold])

    X_train = X[train_indices]
    y_train = y[train_indices]
    X_dev = X[dev_indices]
    y_dev = y[dev_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]

    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_dev = le.transform(y_dev)
    y_test = le.transform(y_test)

    return X_train, y_train, X_dev, y_dev, X_test, y_test

X_train, y_train, X_val, y_dev, X_test, y_test = get_train_val_test_split(folds, test_fold=1, dev_fold=2)
train_dataset = AudioDataset(file_paths=X_train, labels=y_train)
val_dataset = AudioDataset(file_paths=X_val, labels=y_dev)
test_dataset = AudioDataset(file_paths=X_test, labels=y_test)
print(train_dataset.file_paths)
print("Exemplo de carregamento do primeiro dado no train_dataset:")
data, label = train_dataset[0]
print(f"Formato dos dados: {data.shape}")
print(f"Label: {label}")

['processed_audios/fold3/102105-3-0-0.npy'
 'processed_audios/fold3/103199-4-0-0.npy'
 'processed_audios/fold3/103199-4-0-3.npy' ...
 'processed_audios/fold10/99192-4-0-54.npy'
 'processed_audios/fold10/99192-4-0-55.npy'
 'processed_audios/fold10/99192-4-0-7.npy']
Exemplo de carregamento do primeiro dado no train_dataset:
Formato dos dados: torch.Size([1, 32, 376])
Label: 3


###Definição do modelo CNN

Após alguma pesquisa, decidimos criar um modelo com 4 camadas, uma camada convulsional, uma camada pooling e uma camada totalmente ligada. Não existe uma camada softmax para a saída pois iria interferir com o cálculo da cross-entropy-loss durante o treino. O que fizemos foi não definir a camada diretamente no modelo e depois transformar os outputs do modelo em labels "manualmente" antes de testar.
O número de filtros, e de mapas de características (consequentemente) foi definido como 16 na primeira camada, uma escolha conservadora que permite ao modelo correr mais rápido, mas que desacelera um pouco o ritmo de aprendizagem.
"A CNN typically has three layers: a convolutional layer, a pooling layer, and a fully connected layer"  
"Since convolution is a linear operation and images are far from linear, non-linearity layers are often placed directly after the convolutional layer to introduce non-linearity to the activation map"



In [ ]:
class CNNNetwork(nn.Module):
    def __init__(self):
        super(CNNNetwork, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.flatten = nn.Flatten()

        dummy_input = torch.randn(1, 1, 32, 376)
        dummy_output = self.conv1(dummy_input)
        dummy_output = self.conv2(dummy_output)
        dummy_output = self.conv3(dummy_output)
        output_size = dummy_output.numel()

        self.linear = nn.Linear(output_size, 10)

    def forward(self, input_data):
        x = self.conv1(input_data)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        logits = self.linear(x)
        return logits

###Inicialização dos pesos

Inicializamos sempre os pesos do modelo com uma função gaussiana. O bias é inicializado com o valor zero

In [ ]:
def initialize_weights(layer):
    if isinstance(layer, (nn.Conv2d, nn.Linear)):
        mean = 0.0
        stddev = 0.01
        with torch.no_grad():
            layer.weight = nn.Parameter(torch.tensor(np.random.normal(loc=mean, scale=stddev, size=layer.weight.shape), dtype=torch.float32))
        if layer.bias is not None:
            nn.init.zeros_(layer.bias)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

###Otimização Bayesiana

Nesta função corre-se uma otimização bayesiana dos hiperparâmetros de learning rate e nº de epochs (decidimos por um número fixo e razoável de batch size para não prejudicar ainda mais o tempo de execução). A otimização é baseada na loss (pensámos que era a melhor métrica, visto que a accuracy sem 10-cross-validation pode ser mais enganadora). O learning rate tem um scheduler que diminui em 10% o learning rate em cada epoch (para evitar overfitting). Esta otimização executa 20 tentativas para tentar encontrar os melhores parâmetros. No fim de cada epoch utiliza o fold de desenvolvimento para melhorar o treino do modelo e no fim de cada tentativa utiliza o fold de teste para testar o modelo treinado. Retorna os parâmetros do melhor modelo para depois se executar uma 10-cross-validation do modelo com esses parâmetros

In [ ]:
def bayesian_optimization_loss(model_class):
    batch_size = 64

    def objective(trial):
        lr = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
        epochs = trial.suggest_int("epochs", 5, 30, log=True)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        model = model_class()
        model.apply(initialize_weights)
        model.to(device)

        optimizer = optim.Adam(model.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(epochs):
            print(f"Epoch {epoch + 1}/{epochs}: Training...")
            model.train()
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            scheduler.step()

            print(f"Epoch {epoch + 1}/{epochs}: Validation...")
            model.eval()
            dev_loss = 0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    dev_loss += loss.item()

            avg_dev_loss = dev_loss / len(val_loader)
            print(f"Validation Loss: {avg_dev_loss}")

        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        model.eval()
        all_labels = []
        all_predictions = []
        test_loss = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_predictions)
        avg_test_loss = test_loss / len(test_loader)
        cm = confusion_matrix(all_labels, all_predictions)
        cm_percentage = cm / np.sum(cm) * 100

        print("Teste Final após tentativa de otimização:")
        print(f"Test Loss: {avg_test_loss}")
        print(f"Accuracy: {accuracy}")
        print(f"Confusion Matrix:\n{cm_percentage}")

        return avg_test_loss

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)

    best_hyperparams = study.best_params
    best_learning_rate = best_hyperparams["learning_rate"]
    best_epochs = best_hyperparams["epochs"]

    return model, accuracy, cm_percentage, avg_test_loss, best_learning_rate, best_epochs


In [ ]:
best_model, best_model_accuracy, best_model_cfm, best_model_loss, best_model_lr, best_model_epochs = bayesian_optimization_loss(CNNNetwork)

print(f"Best Model Learning Rate: {best_model_lr}")
print(f"Best Model Epochs: {best_model_epochs}")
print(f"Best Model Accuracy: {best_model_accuracy}")
print(f"Best Model Confusion Matrix Percentage:\n{best_model_cfm}")
print(f"Best Model Loss: {best_model_loss}")



[I 2024-11-21 12:31:08,838] A new study created in memory with name: no-name-9970fbc3-22a2-4621-bddb-00556fe6f675


Epoch 1/7: Training...
Epoch 1/7: Validation...
Validation Loss: 2.2362046922956194
Epoch 2/7: Training...
Epoch 2/7: Validation...
Validation Loss: 1.917687680040087
Epoch 3/7: Training...
Epoch 3/7: Validation...
Validation Loss: 1.781204981463296
Epoch 4/7: Training...
Epoch 4/7: Validation...
Validation Loss: 1.757117211818695
Epoch 5/7: Training...
Epoch 5/7: Validation...
Validation Loss: 1.7325877121516637
Epoch 6/7: Training...
Epoch 6/7: Validation...
Validation Loss: 1.6896797588893346
Epoch 7/7: Training...
Epoch 7/7: Validation...
Validation Loss: 1.6827741776193892


[I 2024-11-21 12:41:51,152] Trial 0 finished with value: 1.6396572078977312 and parameters: {'learning_rate': 0.00015560026163806526, 'epochs': 7}. Best is trial 0 with value: 1.6396572078977312.


Teste Final após tentativa de otimização:
Test Loss: 1.6396572078977312
Accuracy: 0.422680412371134
Confusion Matrix:
[[4.12371134 0.         0.68728522 0.         0.         5.8419244
  0.         0.22909507 0.         0.57273769]
 [0.         0.11454754 1.37457045 0.80183276 1.37457045 0.
  0.22909507 0.         0.22909507 0.        ]
 [0.45819015 0.         7.10194731 1.14547537 0.11454754 1.37457045
  0.22909507 0.68728522 0.22909507 0.11454754]
 [1.03092784 0.22909507 0.57273769 7.44558992 1.03092784 0.68728522
  0.11454754 0.11454754 0.11454754 0.11454754]
 [0.         0.         0.57273769 0.22909507 5.04009164 0.
  0.34364261 4.23825888 0.         1.03092784]
 [1.26002291 0.         0.11454754 0.         0.         5.8419244
  0.         0.22909507 3.43642612 0.11454754]
 [0.         0.         0.         0.9163803  1.03092784 0.
  2.06185567 0.         0.         0.        ]
 [0.11454754 0.         6.41466208 0.         0.11454754 1.03092784
  0.         6.07101947 0.         

[I 2024-11-21 12:50:10,031] Trial 1 finished with value: 1.4426610597542353 and parameters: {'learning_rate': 0.0006182160536777261, 'epochs': 7}. Best is trial 1 with value: 1.4426610597542353.


...

Teste Final após tentativa de otimização:
Test Loss: 1.6393347041947501
Accuracy: 0.4043528064146621
Confusion Matrix:
[[3.66552119 0.         0.11454754 0.         0.34364261 6.64375716
  0.         0.34364261 0.         0.34364261]
 [0.         1.37457045 0.34364261 0.34364261 1.37457045 0.
  0.34364261 0.         0.         0.34364261]
 [0.22909507 0.11454754 3.09278351 1.37457045 1.37457045 2.52004582
  0.22909507 0.57273769 1.26002291 0.68728522]
 [0.11454754 0.34364261 0.22909507 6.7583047  0.68728522 1.8327606
  0.34364261 0.45819015 0.22909507 0.45819015]
 [0.         0.34364261 0.         0.11454754 4.35280641 0.11454754
  1.8327606  4.35280641 0.         0.34364261]
 [0.         0.         0.11454754 0.11454754 0.22909507 6.7583047
  0.         0.22909507 3.32187858 0.22909507]
 [0.         0.         0.         1.37457045 0.9163803  0.
  1.71821306 0.         0.         0.        ]
 [0.         0.         0.80183276 0.11454754 0.45819015 4.46735395
  0.11454754 7.21649485 0.

[I 2024-11-21 13:07:43,352] Trial 3 finished with value: 1.4228174558707647 and parameters: {'learning_rate': 0.0003869159086377125, 'epochs': 7}. Best is trial 3 with value: 1.4228174558707647.


Teste Final após tentativa de otimização:
Test Loss: 1.4228174558707647
Accuracy: 0.5395189003436426
Confusion Matrix:
[[6.64375716 0.         1.37457045 0.         0.         2.06185567
  0.         1.14547537 0.         0.22909507]
 [0.         0.11454754 1.26002291 0.57273769 1.60366552 0.
  0.22909507 0.         0.22909507 0.11454754]
 [0.45819015 0.11454754 7.90378007 0.9163803  0.11454754 0.34364261
  0.11454754 0.68728522 0.34364261 0.45819015]
 [0.80183276 0.22909507 1.03092784 7.56013746 0.80183276 0.22909507
  0.         0.22909507 0.45819015 0.11454754]
 [0.80183276 0.         0.57273769 0.         6.41466208 0.
  0.         1.71821306 0.         1.94730813]
 [2.52004582 0.         0.80183276 0.11454754 0.         3.89461627
  0.         0.22909507 3.09278351 0.34364261]
 [0.         0.         0.         1.26002291 0.9163803  0.
  1.8327606  0.         0.         0.        ]
 [0.11454754 0.         4.12371134 0.         0.11454754 1.8327606
  0.         7.56013746 0.       

[I 2024-11-21 13:33:40,529] Trial 4 finished with value: 1.9800939389637537 and parameters: {'learning_rate': 1.597548005475506e-05, 'epochs': 20}. Best is trial 3 with value: 1.4228174558707647.


Teste Final após tentativa de otimização:
Test Loss: 1.9800939389637537
Accuracy: 0.27949599083619703
Confusion Matrix:
[[4.69644903 0.         0.34364261 0.68728522 0.22909507 3.43642612
  0.11454754 0.11454754 0.9163803  0.9163803 ]
 [0.         0.         0.9163803  0.11454754 2.40549828 0.
  0.         0.         0.68728522 0.        ]
 [0.22909507 0.         1.26002291 1.48911798 0.9163803  0.45819015
  0.22909507 0.         4.9255441  1.94730813]
 [3.20733104 0.         0.22909507 1.26002291 2.29095074 0.57273769
  1.14547537 0.         1.94730813 0.80183276]
 [0.34364261 0.         0.         0.45819015 6.98739977 0.
  0.80183276 2.52004582 0.         0.34364261]
 [1.94730813 0.         0.         0.         0.         3.20733104
  0.11454754 0.         5.61282932 0.11454754]
 [0.         0.         0.         0.         1.26002291 0.
  2.17640321 0.11454754 0.         0.45819015]
 [0.         0.         2.52004582 0.45819015 1.71821306 1.14547537
  0.80183276 1.14547537 5.84192

###Treino e teste do modelo com os melhores hiperparâmetros utilizando 10-cross-validation

In [ ]:
num_folds = 10
all_accuracies = []
all_confusion_matrices = []
all_losses = []
model_class = CNNNetwork
lr = best_model_lr
epochs = best_model_epochs
batch_size = 64
kfold = list(range(num_folds))
all_results = []

for test_fold in kfold:
    for dev_fold in kfold:
        if dev_fold == test_fold:
            continue
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        model = model_class()
        model.apply(initialize_weights)
        model.to(device)

        optimizer = optim.Adam(model.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(epochs):
            print(f"Epoch {epoch + 1}/{epochs}: Training...")
            model.train()
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            scheduler.step()

            print(f"Epoch {epoch + 1}/{epochs}: Validation...")
            model.eval()
            dev_loss = 0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    dev_loss += loss.item()

            avg_dev_loss = dev_loss / len(val_loader)
            print(f"Validation Loss: {avg_dev_loss}")

        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        model.eval()
        all_labels = []
        all_predictions = []
        test_loss = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_predictions)
        avg_test_loss = test_loss / len(test_loader)
        cm = confusion_matrix(all_labels, all_predictions)
        cm_percentage = cm / np.sum(cm) * 100

        all_accuracies.append(accuracy)
        all_confusion_matrices.append(cm_percentage)
        all_losses.append(avg_test_loss)

print(f"Accuracy: {np.mean(all_accuracies)}")
print(f"Confusion Matrix: {np.sum(all_confusion_matrices, axis=0)}")
mean_loss = (f"Loss: {np.mean(all_losses)}")

###Referências

https://towardsdatascience.com/convolutional-neural-networks-explained-9cc5188c4939  
https://www.geeksforgeeks.org/how-to-decide-number-of-filters-in-cnn/  
https://www.sciencedirect.com/topics/earth-and-planetary-sciences/bayesian-optimization